**Set environment**

In [2]:
### set env
source ../config/config_duke.sh
source ../config/config_load_module_bedtools.sh
show_env

You are on Duke Server: HARDAC
BASE DIRECTORY:     /gpfs/fs1/data/reddylab/Kuei
PATH OF SOURCE:     /gpfs/fs1/data/reddylab/Kuei/source
PATH OF EXECUTABLE: /gpfs/fs1/data/reddylab/Kuei/exe
PATH OF ANNOTATION: /gpfs/fs1/data/reddylab/Kuei/annotation
PATH OF PROJECT:    /gpfs/fs1/data/reddylab/Kuei/GitRepo/Proj_CombEffect_ENCODE_FCC/notebooks
PATH OF RESULTS:    /gpfs/fs1/data/reddylab/Kuei/out/proj_combeffect_encode_fcc



In [4]:
which bedtools
bedtools -version

/nfs/software/helmod/apps/Core/bedtools2/2.25.0-fasrc01/bin/bedtools
bedtools v2.25.0


**Check data directory**

In [7]:
ls ${FD_RES}/KS91_K562_ASTARRseq/fragment/*unstranded.bed | xargs -n 1 basename

KS91_K562_hg38_ASTARRseq_Input_rep1.FADS.unstranded.bed
KS91_K562_hg38_ASTARRseq_Input_rep1.GATA1.unstranded.bed
KS91_K562_hg38_ASTARRseq_Input_rep1.MYC.unstranded.bed
KS91_K562_hg38_ASTARRseq_Input_rep2.FADS.unstranded.bed
KS91_K562_hg38_ASTARRseq_Input_rep2.GATA1.unstranded.bed
KS91_K562_hg38_ASTARRseq_Input_rep2.MYC.unstranded.bed
KS91_K562_hg38_ASTARRseq_Input_rep3.FADS.unstranded.bed
KS91_K562_hg38_ASTARRseq_Input_rep3.GATA1.unstranded.bed
KS91_K562_hg38_ASTARRseq_Input_rep3.MYC.unstranded.bed
KS91_K562_hg38_ASTARRseq_Input_rep4.FADS.unstranded.bed
KS91_K562_hg38_ASTARRseq_Input_rep4.GATA1.unstranded.bed
KS91_K562_hg38_ASTARRseq_Input_rep4.MYC.unstranded.bed
KS91_K562_hg38_ASTARRseq_Input_rep5.FADS.unstranded.bed
KS91_K562_hg38_ASTARRseq_Input_rep5.GATA1.unstranded.bed
KS91_K562_hg38_ASTARRseq_Input_rep5.MYC.unstranded.bed
KS91_K562_hg38_ASTARRseq_Input_rep6.FADS.unstranded.bed
KS91_K562_hg38_ASTARRseq_Input_rep6.GATA1.unstranded.bed
KS91_K562_hg38_ASTARRseq_Input_rep6.MYC.unstran

In [8]:
ls ${FD_RES}/KS91_K562_ASTARRseq/fragment/*unstranded.bed | xargs -n 1 basename | wc -l

30


In [11]:
ls ${FD_RES}/KS91_K562_ASTARRseq/fragment/*GATA1.unstranded.bed | xargs -n 1 basename | wc -l
ls ${FD_RES}/KS91_K562_ASTARRseq/fragment/*MYC.unstranded.bed | xargs -n 1 basename | wc -l
ls ${FD_RES}/KS91_K562_ASTARRseq/fragment/*FADS.unstranded.bed | xargs -n 1 basename | wc -l

10
10
10


**Check chromosome**

In [16]:
FDIRY=${FD_RES}/KS91_K562_ASTARRseq/fragment

FNAME=KS91_K562_hg38_ASTARRseq_Input_rep1.GATA1.unstranded.bed
head -1 ${FDIRY}/${FNAME}

FNAME=KS91_K562_hg38_ASTARRseq_Input_rep1.MYC.unstranded.bed
head -1 ${FDIRY}/${FNAME}

FNAME=KS91_K562_hg38_ASTARRseq_Input_rep1.FADS.unstranded.bed
head -1 ${FDIRY}/${FNAME}

chrX	47787165	47787363	chrX_47787165_47787363	1	.
chr8	126735990	126736404	chr8_126735990_126736404	1	.
chr11	61787228	61787382	chr11_61787228_61787382	1	.


```
GATA1 --- chrX
MYC   --- chr8
FADS  --- chr11
```

## Calculate NUC for fragments in ATAC-STARR

### Region: GATA

**RUN**

In [21]:
sbatch -p ${NODE} \
    -J bedtools_nuc_astarr_gata1 \
    --mem 20G \
    --array 0-9 \
    -o ${FD_LOG}/fragment_nuc_astarr_gata1.%a.txt \
    <<'EOF'
#!/bin/bash
### set directories & global variables
source ../config/config_duke.sh
CHROM=chrX
REGION=GATA1

### set input and output
FD_BED=${FD_RES}/KS91_K562_ASTARRseq/fragment
FP_BEDS=($(ls ${FD_BED}/*${REGION}.unstranded.bed))
FP_BED=${FP_BEDS[${SLURM_ARRAY_TASK_ID}]}
FN_BED=$(basename ${FP_BED})

FD_OUT=${FD_RES}/KS91_K562_ASTARRseq/fragment_nuc
FN_OUT=${FN_BED}.gz

### genome annotation
FD_GEN=${FD_ANN}/genome/hg38/fasta
FN_GEN=${CHROM}.fa

### print start message
timer_start=`date +%s`
echo "Hostname:          " $(hostname)
echo "Slurm Array Index: " ${SLURM_ARRAY_TASK_ID}
echo "Time Stamp:        " $(date +"%m-%d-%y+%T")
echo

### Show input/output file
echo "Input  file: " ${FD_BED}/${FN_BED}
echo "Output file: " ${FD_OUT}/${FN_OUT}
echo
echo "Show the first few lines of the input file"
echo ${FD_BED}/${FN_BED}
cat  ${FD_BED}/${FN_BED} | head -n 3 
echo
    
### init: create output folder if not exist
mkdir -p ${FD_OUT}

### calculate the GC content of each fragment
bedtools nuc \
    -fi  ${FD_GEN}/${FN_GEN} \
    -bed ${FD_BED}/${FN_BED} |\
    gzip -c > ${FD_OUT}/${FN_OUT}

### Show input/output file
echo "Show the first few lines of the output file"
echo ${FD_OUT}/${FN_OUT}
zcat ${FD_OUT}/${FN_OUT} | head -n 3
echo

### print end message
timer_stop=`date +%s`
runtime=$(echo "${timer_stop} - ${timer_start}" | bc -l)
echo
echo 'Done!'
echo "Run Time: $(displaytime ${runtime})"

EOF

Submitted batch job 29147777


**CHECK**

In [22]:
cat ${FD_LOG}/fragment_nuc_astarr_gata1.0.txt

Hostname:           dl-01.genome.duke.edu
Slurm Array Index:  0
Time Stamp:         10-11-22+18:22:46

Input  file:  /gpfs/fs1/data/reddylab/Kuei/out/proj_combeffect_encode_fcc/KS91_K562_ASTARRseq/fragment/KS91_K562_hg38_ASTARRseq_Input_rep1.GATA1.unstranded.bed
Output file:  /gpfs/fs1/data/reddylab/Kuei/out/proj_combeffect_encode_fcc/KS91_K562_ASTARRseq/fragment_nuc/KS91_K562_hg38_ASTARRseq_Input_rep1.GATA1.unstranded.bed.gz

Show the first few lines of the input file
/gpfs/fs1/data/reddylab/Kuei/out/proj_combeffect_encode_fcc/KS91_K562_ASTARRseq/fragment/KS91_K562_hg38_ASTARRseq_Input_rep1.GATA1.unstranded.bed
chrX	47787165	47787363	chrX_47787165_47787363	1	.
chrX	47787557	47787773	chrX_47787557_47787773	1	.
chrX	47787593	47787783	chrX_47787593_47787783	1	.

Show the first few lines of the output file
/gpfs/fs1/data/reddylab/Kuei/out/proj_combeffect_encode_fcc/KS91_K562_ASTARRseq/fragment_nuc/KS91_K562_hg38_ASTARRseq_Input_rep1.GATA1.unstranded.bed.gz
#1_usercol	2_usercol	3_usercol	4

### Region: MYC

**RUN**

In [23]:
sbatch -p ${NODE} \
    -J bedtools_nuc_astarr_myc \
    --mem 20G \
    --array 0-9 \
    -o ${FD_LOG}/fragment_nuc_astarr_myc.%a.txt \
    <<'EOF'
#!/bin/bash
### set directories & global variables
source ../config/config_duke.sh
CHROM=chr8
REGION=MYC

### set input and output
FD_BED=${FD_RES}/KS91_K562_ASTARRseq/fragment
FP_BEDS=($(ls ${FD_BED}/*${REGION}.unstranded.bed))
FP_BED=${FP_BEDS[${SLURM_ARRAY_TASK_ID}]}
FN_BED=$(basename ${FP_BED})

FD_OUT=${FD_RES}/KS91_K562_ASTARRseq/fragment_nuc
FN_OUT=${FN_BED}.gz

### genome annotation
FD_GEN=${FD_ANN}/genome/hg38/fasta
FN_GEN=${CHROM}.fa

### print start message
timer_start=`date +%s`
echo "Hostname:          " $(hostname)
echo "Slurm Array Index: " ${SLURM_ARRAY_TASK_ID}
echo "Time Stamp:        " $(date +"%m-%d-%y+%T")
echo

### Show input/output file
echo "Input  file: " ${FD_BED}/${FN_BED}
echo "Output file: " ${FD_OUT}/${FN_OUT}
echo
echo "Show the first few lines of the input file"
echo ${FD_BED}/${FN_BED}
cat  ${FD_BED}/${FN_BED} | head -n 3 
echo
    
### init: create output folder if not exist
mkdir -p ${FD_OUT}

### calculate the GC content of each fragment
bedtools nuc \
    -fi  ${FD_GEN}/${FN_GEN} \
    -bed ${FD_BED}/${FN_BED} |\
    gzip -c > ${FD_OUT}/${FN_OUT}

### Show input/output file
echo "Show the first few lines of the output file"
echo ${FD_OUT}/${FN_OUT}
zcat ${FD_OUT}/${FN_OUT} | head -n 3
echo

### print end message
timer_stop=`date +%s`
runtime=$(echo "${timer_stop} - ${timer_start}" | bc -l)
echo
echo 'Done!'
echo "Run Time: $(displaytime ${runtime})"

EOF

Submitted batch job 29147787


**CHECK**

In [26]:
cat ${FD_LOG}/fragment_nuc_astarr_myc.0.txt

Hostname:           dl-01.genome.duke.edu
Slurm Array Index:  0
Time Stamp:         10-11-22+18:25:16

Input  file:  /gpfs/fs1/data/reddylab/Kuei/out/proj_combeffect_encode_fcc/KS91_K562_ASTARRseq/fragment/KS91_K562_hg38_ASTARRseq_Input_rep1.MYC.unstranded.bed
Output file:  /gpfs/fs1/data/reddylab/Kuei/out/proj_combeffect_encode_fcc/KS91_K562_ASTARRseq/fragment_nuc/KS91_K562_hg38_ASTARRseq_Input_rep1.MYC.unstranded.bed.gz

Show the first few lines of the input file
/gpfs/fs1/data/reddylab/Kuei/out/proj_combeffect_encode_fcc/KS91_K562_ASTARRseq/fragment/KS91_K562_hg38_ASTARRseq_Input_rep1.MYC.unstranded.bed
chr8	126735990	126736404	chr8_126735990_126736404	1	.
chr8	126736004	126736113	chr8_126736004_126736113	1	.
chr8	126736010	126736129	chr8_126736010_126736129	1	.

Show the first few lines of the output file
/gpfs/fs1/data/reddylab/Kuei/out/proj_combeffect_encode_fcc/KS91_K562_ASTARRseq/fragment_nuc/KS91_K562_hg38_ASTARRseq_Input_rep1.MYC.unstranded.bed.gz
#1_usercol	2_usercol	3_userc

### Region: FADS

**RUN**

In [24]:
sbatch -p ${NODE} \
    -J bedtools_nuc_astarr_fads \
    --mem 20G \
    --array 0-9 \
    -o ${FD_LOG}/fragment_nuc_astarr_fads.%a.txt \
    <<'EOF'
#!/bin/bash
### set directories & global variables
source ../config/config_duke.sh
CHROM=chr11
REGION=FADS

### set input and output
FD_BED=${FD_RES}/KS91_K562_ASTARRseq/fragment
FP_BEDS=($(ls ${FD_BED}/*${REGION}.unstranded.bed))
FP_BED=${FP_BEDS[${SLURM_ARRAY_TASK_ID}]}
FN_BED=$(basename ${FP_BED})

FD_OUT=${FD_RES}/KS91_K562_ASTARRseq/fragment_nuc
FN_OUT=${FN_BED}.gz

### genome annotation
FD_GEN=${FD_ANN}/genome/hg38/fasta
FN_GEN=${CHROM}.fa

### print start message
timer_start=`date +%s`
echo "Hostname:          " $(hostname)
echo "Slurm Array Index: " ${SLURM_ARRAY_TASK_ID}
echo "Time Stamp:        " $(date +"%m-%d-%y+%T")
echo

### Show input/output file
echo "Input  file: " ${FD_BED}/${FN_BED}
echo "Output file: " ${FD_OUT}/${FN_OUT}
echo
echo "Show the first few lines of the input file"
echo ${FD_BED}/${FN_BED}
cat  ${FD_BED}/${FN_BED} | head -n 3 
echo
    
### init: create output folder if not exist
mkdir -p ${FD_OUT}

### calculate the GC content of each fragment
bedtools nuc \
    -fi  ${FD_GEN}/${FN_GEN} \
    -bed ${FD_BED}/${FN_BED} |\
    gzip -c > ${FD_OUT}/${FN_OUT}

### Show input/output file
echo "Show the first few lines of the output file"
echo ${FD_OUT}/${FN_OUT}
zcat ${FD_OUT}/${FN_OUT} | head -n 3
echo

### print end message
timer_stop=`date +%s`
runtime=$(echo "${timer_stop} - ${timer_start}" | bc -l)
echo
echo 'Done!'
echo "Run Time: $(displaytime ${runtime})"

EOF

Submitted batch job 29147797


**CHECK**

In [27]:
cat ${FD_LOG}/fragment_nuc_astarr_fads.0.txt

Hostname:           dl-01.genome.duke.edu
Slurm Array Index:  0
Time Stamp:         10-11-22+18:25:46

Input  file:  /gpfs/fs1/data/reddylab/Kuei/out/proj_combeffect_encode_fcc/KS91_K562_ASTARRseq/fragment/KS91_K562_hg38_ASTARRseq_Input_rep1.FADS.unstranded.bed
Output file:  /gpfs/fs1/data/reddylab/Kuei/out/proj_combeffect_encode_fcc/KS91_K562_ASTARRseq/fragment_nuc/KS91_K562_hg38_ASTARRseq_Input_rep1.FADS.unstranded.bed.gz

Show the first few lines of the input file
/gpfs/fs1/data/reddylab/Kuei/out/proj_combeffect_encode_fcc/KS91_K562_ASTARRseq/fragment/KS91_K562_hg38_ASTARRseq_Input_rep1.FADS.unstranded.bed
chr11	61787228	61787382	chr11_61787228_61787382	1	.
chr11	61787230	61787628	chr11_61787230_61787628	1	.
chr11	61787234	61787471	chr11_61787234_61787471	1	.

Show the first few lines of the output file
/gpfs/fs1/data/reddylab/Kuei/out/proj_combeffect_encode_fcc/KS91_K562_ASTARRseq/fragment_nuc/KS91_K562_hg38_ASTARRseq_Input_rep1.FADS.unstranded.bed.gz
#1_usercol	2_usercol	3_usercol